In [65]:
from pathlib import Path
import datetime as dt

import echopype as ep
import gen_mvbs_utils

In [27]:
# Parameters
output_path = Path("./tmp_outputs")
start_datetime = dt.datetime(2015, 8, 30, 0, 0)
end_datetime = dt.datetime(2015, 9, 1, 23, 59, 59)

In [28]:
if not output_path.exists():
    output_path.mkdir()

In [29]:
ooi_site_url = (
    "https://rawdata.oceanobservatories.org/files/"
    "CE04OSPS/PC01B/ZPLSCB102_10.33.10.143/"
)


In [30]:
desired_raw_file_urls = gen_mvbs_utils.get_raw_file_url(
    file_url=ooi_site_url, start=start_datetime, end=end_datetime
)

In [31]:
print(f"There are {len(desired_raw_file_urls)} raw files within the specified datetime range.")

There are 3 raw files within the specified datetime range.


In [32]:
MVBS_range_interval_meter = 5
MVBS_ping_time_interval_seconds = "200S"
MVBS_str = f"mvbs_{MVBS_range_interval_meter}m_{MVBS_ping_time_interval_seconds[:-1]}s"
MVBS_str

'mvbs_5m_200s'

In [52]:
%%time
for raw_file_url in desired_raw_file_urls:
    raw_path = Path(raw_file_url)
    ed = ep.open_raw(raw_file=raw_path, sonar_model='ek60')
    ds_Sv = ep.calibrate.compute_Sv(ed)
    ds_MVBS = ep.preprocess.compute_MVBS(
        ds_Sv=ds_Sv,
        range_meter_bin=MVBS_range_interval_meter,
        ping_time_bin=MVBS_ping_time_interval_seconds
    )
    ds_MVBS.to_zarr(
        output_path / raw_path.with_name(raw_path.stem + "_MVBS").with_suffix(".zarr").name
        )

22:44:04  parsing file OOI-D20150830-T000000.raw, time of first ping: 2015-Aug-30 00:00:00
22:44:44  parsing file OOI-D20150831-T000000.raw, time of first ping: 2015-Aug-31 00:00:00
22:45:24  parsing file OOI-D20150901-T000000.raw, time of first ping: 2015-Sep-01 00:00:00
CPU times: user 53.8 s, sys: 25.6 s, total: 1min 19s
Wall time: 2min 1s
